In [1]:
import json
import re

In [89]:


with open("bin/quotesRaw.txt", "r") as f:
    data = f.read()

j = json.loads(data)
with open("bin/quotesRaw-replaced.json", "w") as f:
     json.dump(j, f, indent=2)

# garbage collect j, data
del j
del data

In [2]:
with open("bin/quotesRaw-replaced.json", "r") as f:
    data = json.load(f)

print("The length of the payload data is:", len(data["payload"]))

The length of the payload data is: 1600


In [21]:
payload = data["payload"]

# login attempt
print("Login attempt:", payload[0][:100], "[REDACTED] ...\n")

# first quote
print("First quote:", payload[1][:300], "...", payload[1][-300:])

Login attempt: [{"body":{"patches":[{"op":"replace","path":"","value":{"authenticated":true,"authenticationStatus": [REDACTED] ...

First quote: [{"body":{"patches":[{"op":"replace","path":"","value":{"exchanges":["AMEX","BATS","BEST","BOX","C2","CBOE","ISE","NASDAQ","NYSE","PHLX"],"items":[{"symbol":".SPY210616C230","values":{"PROBABILITY_ITM":1}},{"symbol":".SPY210616P230","values":{"PROBABILITY_ITM":0}},{"symbol":".SPY210616C235","values" ... P 21 100 (Quarterlys) 15 OCT 21 100 19 NOV 21 100 17 DEC 21 100 31 DEC 21 100 (Quarterlys) 21 JAN 22 100 18 FEB 22 100 18 MAR 22 100 31 MAR 22 100 (Quarterlys) 17 JUN 22 100 16 SEP 22 100 16 DEC 22 100 20 JAN 23 100 17 MAR 23 100 15 DEC 23 100]-0","service":"quotes/options","type":"patch","ver":0}}]


In [4]:
# parse these patches into individual items and delete the main package
packages = []
for item in payload:
    item_json = json.loads(item)
    for ele in item_json:
        for patch in ele["body"]["patches"]:
            tup = (patch, ele["header"]["id"])
            packages.append(tup)

print(f"The length of the packages list is: {len(packages):,}")

The length of the packages list is: 6,704,490


In [23]:
# first quote (first and last 300 characters shows tuple form)
print("First quote:", str(packages[1])[:300], "\n..."*3, str(packages[1])[-500:] + "\n")

# second quote
print("Second quote:", str(packages[2])[:300], "\n..."*3, str(packages[2])[-500:] + "\n")

First quote: ({'op': 'replace', 'path': '', 'value': {'exchanges': ['AMEX', 'BATS', 'BEST', 'BOX', 'C2', 'CBOE', 'ISE', 'NASDAQ', 'NYSE', 'PHLX'], 'items': [{'symbol': '.SPY210616C230', 'values': {'PROBABILITY_ITM': 1}}, {'symbol': '.SPY210616P230', 'values': {'PROBABILITY_ITM': 0}}, {'symbol': '.SPY210616C235', 
...
...
... ) 6 JUL 21 100 7 JUL 21 100 (Weeklys) 9 JUL 21 100 (Weeklys) 12 JUL 21 100 (Weeklys) 14 JUL 21 100 (Weeklys) 16 JUL 21 100 19 JUL 21 100 (Weeklys) 21 JUL 21 100 (Weeklys) 23 JUL 21 100 (Weeklys) 30 JUL 21 100 (Weeklys) 20 AUG 21 100 17 SEP 21 100 30 SEP 21 100 (Quarterlys) 15 OCT 21 100 19 NOV 21 100 17 DEC 21 100 31 DEC 21 100 (Quarterlys) 21 JAN 22 100 18 FEB 22 100 18 MAR 22 100 31 MAR 22 100 (Quarterlys) 17 JUN 22 100 16 SEP 22 100 16 DEC 22 100 20 JAN 23 100 17 MAR 23 100 15 DEC 23 100]-0')

Second quote: ({'op': 'replace', 'path': '', 'value': {'exchanges': ['AMEX', 'BATS', 'BEST', 'BOX', 'C2', 'CBOE', 'ISE', 'NASDAQ', 'NYSE', 'PHLX'], 'items': [{'symbol': '.

In [5]:
patch_by_id = {}
for patch, id in packages:
    if id not in patch_by_id:
        patch_by_id[id] = []
    patch_by_id[id].append(patch)

print("PATCHES BY TICKER" + "\n" + "-"*30)
patch_by_id_count = {k[:20]:len(v) for k,v in patch_by_id.items()}
for k, v in {k: v for k, v in sorted(patch_by_id_count.items(), reverse=True, key=lambda item: item[1])}.items():
    print(k[:k.find("@")], "-", f"{v:,}")


PATCHES BY TICKER
------------------------------
OPTIONCHAINGET#SPY - 3,455,220
OPTIONCHAINGET#TSLA - 1,766,005
OPTIONCHAINGET#GLD - 548,102
OPTIONCHAINGET#AMC - 434,721
OPTIONCHAINGET#AAPL - 304,385
OPTIONCHAINGET#NIO - 167,722
OPTIONCHAINGET#WISH - 28,334
logi - 1


In [6]:
# let us pick SPY for research purposes
[spy_patch_key] = [k for k in patch_by_id if k.startswith("OPTIONCHAINGET#SPY")]
print(spy_patch_key)

OPTIONCHAINGET#SPY@[16 JUN 21 100 (Weeklys) 18 JUN 21 100 21 JUN 21 100 (Weeklys) 23 JUN 21 100 (Weeklys) 25 JUN 21 100 (Weeklys) 28 JUN 21 100 (Weeklys) 30 JUN 21 100 (Quarterlys) 2 JUL 21 100 (Weeklys) 6 JUL 21 100 7 JUL 21 100 (Weeklys) 9 JUL 21 100 (Weeklys) 12 JUL 21 100 (Weeklys) 14 JUL 21 100 (Weeklys) 16 JUL 21 100 19 JUL 21 100 (Weeklys) 21 JUL 21 100 (Weeklys) 23 JUL 21 100 (Weeklys) 30 JUL 21 100 (Weeklys) 20 AUG 21 100 17 SEP 21 100 30 SEP 21 100 (Quarterlys) 15 OCT 21 100 19 NOV 21 100 17 DEC 21 100 31 DEC 21 100 (Quarterlys) 21 JAN 22 100 18 FEB 22 100 18 MAR 22 100 31 MAR 22 100 (Quarterlys) 17 JUN 22 100 16 SEP 22 100 16 DEC 22 100 20 JAN 23 100 17 MAR 23 100 15 DEC 23 100]-0


In [7]:
spy_patches = patch_by_id[spy_patch_key]
print(f"Number of patches for SPY: {len(spy_patches):,}")

Number of patches for SPY: 3,455,220


In [8]:
# how many tickers are there in the spy patch list?
print(f"Number of tickers: {len(spy_patches[0]['value']['items']):,}")

Number of tickers: 10,290


In [11]:
# first patch
print("First patch:", str(spy_patches[0])[:300], "\n..."*3, str(spy_patches[0])[-300:] + "\n")

# second patch
print("Second patch:", str(spy_patches[1])[:300], "\n..."*3, str(spy_patches[1])[-300:] + "\n")

# third patch
for i, p in enumerate(spy_patches[2:100000:10000]):
    print(f"Patch {i*10000}:", str(p)[:300])

# Conclusion: we need to keep track of all the tickers and the respective index they're in (unlikely they'd change)

First patch: {'op': 'replace', 'path': '', 'value': {'exchanges': ['AMEX', 'BATS', 'BEST', 'BOX', 'C2', 'CBOE', 'ISE', 'NASDAQ', 'NYSE', 'PHLX'], 'items': [{'symbol': '.SPY210616C230', 'values': {'PROBABILITY_ITM': 1}}, {'symbol': '.SPY210616P230', 'values': {'PROBABILITY_ITM': 0}}, {'symbol': '.SPY210616C235',  
...
...
... 13.91, 'OPEN_INT': 0, 'PROBABILITY_ITM': 1, 'RHO': -15.7246, 'THETA': -0.0134, 'VEGA': 0, 'VOLUME': 0}}, {'symbol': '.SPY231215C635', 'values': {'ASK': 2.36, 'BID': 1.71, 'LAST': 2, 'OPEN_INT': 5, 'PROBABILITY_ITM': 0, 'VOLUME': 53}}, {'symbol': '.SPY231215P635', 'values': {'PROBABILITY_ITM': 1}}]}}

Second patch: {'op': 'replace', 'path': '', 'value': {'exchanges': ['AMEX', 'BATS', 'BEST', 'BOX', 'C2', 'CBOE', 'ISE', 'NASDAQ', 'NYSE', 'PHLX'], 'items': [{'symbol': '.SPY210616C230', 'values': {'ASK': 192.74, 'BID': 191.99, 'DELTA': 0.8937, 'GAMMA': 0, 'IMPLIED_VOLATILITY': -1e+99, 'LAST': 0, 'MARK': 192.31, ' 
...
...
... 'THETA': -0.0053, 'VEGA': 0.6953, 'VOLUME':

In [10]:
# apply main patches which have a path of ""

import jsonpatch

symbol_map = {}
for patch in spy_patches:
    if patch["path"] == "":
        symbol_map = jsonpatch.apply_patch(symbol_map, [patch])

with open("test.json", "w") as f:
    json.dump(symbol_map, f, indent=2)

In [12]:
# seperate patches to their tickers

patch_by_idx = {}
for patch in spy_patches:
    if patch["path"] == "" or not patch["path"].startswith("/items"):
        continue

    # get path without items
    path = patch["path"][7:]

    # get only the index
    idx = int(path[:path.find("/")])
        
    if idx not in patch_by_idx:
        patch_by_idx[idx] = []
    patch_by_idx[idx].append(patch)

In [36]:
# let's view some of these

for idx in list(patch_by_idx.keys())[:5]:
    print(idx)
    for i, patch in enumerate(patch_by_idx[idx][:3]):
        print(f"  | -> ({i + 1})\t", patch)

10285
  | -> (1)	 {'op': 'replace', 'path': '/items/10285/values/ASK', 'value': 211}
  | -> (2)	 {'op': 'replace', 'path': '/items/10285/values/MARK', 'value': 208.97}
  | -> (3)	 {'op': 'replace', 'path': '/items/10285/values/MARK', 'value': 208.99}
10284
  | -> (1)	 {'op': 'replace', 'path': '/items/10284/values/IMPLIED_VOLATILITY', 'value': 0.1568}
  | -> (2)	 {'op': 'replace', 'path': '/items/10284/values/RHO', 'value': 0.5283}
  | -> (3)	 {'op': 'replace', 'path': '/items/10284/values/VEGA', 'value': 0.7534}
10282
  | -> (1)	 {'op': 'replace', 'path': '/items/10282/values/IMPLIED_VOLATILITY', 'value': 0.1517}
  | -> (2)	 {'op': 'replace', 'path': '/items/10282/values/MARK', 'value': 2.07}
  | -> (3)	 {'op': 'replace', 'path': '/items/10282/values/MARK_CHANGE', 'value': -0.32}
10268
  | -> (1)	 {'op': 'replace', 'path': '/items/10268/values/IMPLIED_VOLATILITY', 'value': 0.1534}
  | -> (2)	 {'op': 'replace', 'path': '/items/10268/values/MARK', 'value': 3.75}
  | -> (3)	 {'op': 'repl

In [13]:
# apply these smaller patches (just one for the sake of example) view first

idx_key = list(patch_by_idx.keys())[0]
print("Going to patch", idx_key, "which has", len(patch_by_idx[idx_key]), "items to update")
for i, patch in enumerate(patch_by_idx[idx_key]):
    print(f"| -> ({i + 1})\t", patch)
#symbol_map["items"][]

Going to patch 10285 which has 86 items to update
| -> (1)	 {'op': 'replace', 'path': '/items/10285/values/ASK', 'value': 211}
| -> (2)	 {'op': 'replace', 'path': '/items/10285/values/MARK', 'value': 208.97}
| -> (3)	 {'op': 'replace', 'path': '/items/10285/values/MARK', 'value': 208.99}
| -> (4)	 {'op': 'replace', 'path': '/items/10285/values/ASK', 'value': 210.95}
| -> (5)	 {'op': 'replace', 'path': '/items/10285/values/ASK', 'value': 211}
| -> (6)	 {'op': 'replace', 'path': '/items/10285/values/ASK', 'value': 210.95}
| -> (7)	 {'op': 'replace', 'path': '/items/10285/values/ASK', 'value': 211}
| -> (8)	 {'op': 'replace', 'path': '/items/10285/values/BID', 'value': 207}
| -> (9)	 {'op': 'replace', 'path': '/items/10285/values/BID', 'value': 207.03}
| -> (10)	 {'op': 'replace', 'path': '/items/10285/values/ASK', 'value': 210.95}
| -> (11)	 {'op': 'replace', 'path': '/items/10285/values/ASK', 'value': 211}
| -> (12)	 {'op': 'replace', 'path': '/items/10285/values/MARK', 'value': 209}
| 

In [14]:
# get the last modification to any field (for visual purposes)
last_occ_map = {}
for i, patch in enumerate(patch_by_idx[idx_key]):
    field = patch["path"][patch["path"].rfind("/") + 1:]
    last_occ_map[field] = i

for k, v in last_occ_map.items():
    print(f"{k : <20} {v + 1 : >2}")


ASK                  73
MARK                 84
BID                  86
IMPLIED_VOLATILITY   81
DELTA                77
RHO                  78
THETA                79
VEGA                 80


In [15]:
# let's see how much useless date is given to us :D
comp_c = 0
tot_c = 0
for idx in list(patch_by_idx.keys()):
    last_occ_map = {}
    for i, patch in enumerate(patch_by_idx[idx_key]):
        field = patch["path"][patch["path"].rfind("/") + 1:]
        last_occ_map[field] = i
        tot_c += 1

    comp_c += len(last_occ_map)

print(f"{'USEFUL PATCHES' : <20} {comp_c :>5,}")
print(f"{'GIVEN PATCHES' : <20} {tot_c :>5,}")
print(f"{'FREE UP PERCENT' : <20} {100 - 100*(comp_c/tot_c) :>5,.2f}%")

# lol... that's interesting to say the least

USEFUL PATCHES       82,320
GIVEN PATCHES        884,940
FREE UP PERCENT      90.70%


In [28]:
# apply first patch (followed by more patches afterwards)
idx_key = list(patch_by_idx.keys())[0]
print("Going to patch", idx_key, "which has", len(patch_by_idx[idx_key]), "items to update")

print(f"Currently stored at index {idx_key} is:")
for k, v in symbol_map["items"][idx_key]["values"].items():
    print(f"| -> \t{k : <20} {v + 1 : >0}")

print("Applying patch now...")
#symbol_map["items"][idx_key] = 

patches = []
for patch in patch_by_idx[idx_key]:
    patch["path"] = patch["path"][patch["path"].find("/values"):]
    patches.append(patch)
    print(patch)

symbol_map["items"][idx_key] = jsonpatch.apply_patch(symbol_map["items"][idx_key], patches[::-1])

print(f"Now stored at index {idx_key} is:")
for k, v in symbol_map["items"][idx_key]["values"].items():
    print(f"| -> \t{k : <20} {v + 1 : >0}")
print(f"| ->\t{symbol_map['items'][idx_key]['symbol']}")

Going to patch 10285 which has 86 items to update
Currently stored at index 10285 is:
| -> 	ASK                  212
| -> 	BID                  208.08
| -> 	DELTA                0
| -> 	GAMMA                1
| -> 	IMPLIED_VOLATILITY   1
| -> 	LAST                 216.9
| -> 	MARK                 210.04
| -> 	OPEN_INT             3
| -> 	PROBABILITY_ITM      2
| -> 	RHO                  1
| -> 	THETA                1
| -> 	VEGA                 125.2665
| -> 	VOLUME               1
Applying patch now...
{'op': 'replace', 'path': '/values/ASK', 'value': 211}
{'op': 'replace', 'path': '/values/MARK', 'value': 208.97}
{'op': 'replace', 'path': '/values/MARK', 'value': 208.99}
{'op': 'replace', 'path': '/values/ASK', 'value': 210.95}
{'op': 'replace', 'path': '/values/ASK', 'value': 211}
{'op': 'replace', 'path': '/values/ASK', 'value': 210.95}
{'op': 'replace', 'path': '/values/ASK', 'value': 211}
{'op': 'replace', 'path': '/values/BID', 'value': 207}
{'op': 'replace', 'path': '/values/BID

# General Guideline for Data Pipeline

A single patch comes in which contains:

    -> Request ID (useful for linking this with the requested ticker)
    -> "Patches" (an array of patches that are relevant for this patch)

From this step, we do a few things:

1. Split up all the patches since we know they all belong to the same security, now have: ([] of patches)
2. Go through all the patches (and replace any empty paths first)
3. For all other paths, make a map with k:v being index:[]patch
4. Iterate through this map and apply the patches, export CSV, continue
5. ???
6. Profit
